In [ ]:
# Shared (Client and Server only) Secret Information
IV = '*9wqgg$hsakh8!@v' # multiple of 16 bytes
KEY = 'This is a key123This is a key123' # 16, 24 or 32 bytes
# BLOCK_SIZE = 64 #64

In [ ]:
!pip3 install pycrypto
!pip3 install cryptography

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import datetime
import hashlib
import random
from Crypto.Cipher import AES

BLOCK_SIZE = 16000 # 16000 #16 kb

In [ ]:
# from cryptography.fernet import Fernet
# key = b'51gwkTdL2pP3nJd4Vd7rBoZO301cK0Ga4R7xHg_s1BE='
# # Fernet.generate_key()
# print(type(key), key)

In [ ]:
from base64 import b64encode, b64decode

def encrypt_block(text):
    global KEY
    cipher = AES.new(KEY, AES.MODE_ECB)
    if len(text) %16!=0:
        while not len(text)%16==0:
            text=text+" "
    encrypted_text =cipher.encrypt(text)
    return b64encode(encrypted_text).decode('UTF-8')

def decrypt_block(text):
    global KEY
    text = b64decode(text) # decode before decryption
    decipher = AES.new(KEY, AES.MODE_ECB)
    return decipher.decrypt(text).decode('UTF-8')

In [ ]:
def calcBlockHash(block):
  hasher = hashlib.sha256()
  hasher.update(str(block).encode('utf-8'))
  return hasher.digest()

In [ ]:
def calcFileBlockHashes(encrypted_file_blocks):
  hashed_blocks = []
  for i in encrypted_file_blocks:
    hashed_blocks.append(calcBlockHash(i.encode('utf-8')))
  return hashed_blocks

In [ ]:
# # Instance the Fernet class with the key (AES)
# fernet = Fernet(key)

# def encrypt_block(text):
#   encMessage = fernet.encrypt(text.encode())
#   return encMessage.decode("utf-8")

# def decrypt_block(text):
#   decMessage = fernet.decrypt(text).decode()
#   return decMessage

In [ ]:
# def calcBlockHash(block):
#   hasher = hashlib.sha256()
#   hasher.update(str(block).encode())
#   return hasher.hexdigest()

# def calcFileBlockHashes(encrypted_file_blocks):
#   hashed_blocks = []
#   for i in encrypted_file_blocks:
#     hashed_blocks.append(calcBlockHash(i))
#   return hashed_blocks

In [ ]:
def padding_64(text_file, chunk_size=BLOCK_SIZE):
  chunks = [text_file[i:i+chunk_size] for i in range(0, len(text_file), chunk_size)]

  if len(chunks[-1]) < chunk_size:
    last_chunk = chunks[-1].ljust(chunk_size,'=')
    chunks[-1] = last_chunk

  if len(chunks) % 2 != 0:
    chunks.append("="*chunk_size)

  return chunks

In [ ]:
def calculate_root_from_sib_path(input_hash, sibling_nodes, node_based=False):
    # node_based = True , if working with nodes 
    current_hash = input_hash 
    for i in range(0, len(sibling_nodes)):
      if(sibling_nodes[i][0] == 'left'):
        if node_based is True:
           temp = current_hash
           current_hash = calcBlockHash(sibling_nodes[i][1].data + current_hash) #Hashed
           
           print("LEFT");
           print("node range ",sibling_nodes[i][1].range)
           if(sibling_nodes[i][1].parent is not None): 
            print("parent hash ",sibling_nodes[i][1].parent.data, current_hash)
            print("left hash ",sibling_nodes[i][1].parent.left.data, sibling_nodes[i][1].data)
            print("right hash ",sibling_nodes[i][1].parent.right.data, temp)
           
        else:
          current_hash = calcBlockHash(sibling_nodes[i][1] + current_hash) #Hashed
      else:
        if node_based is True:
          temp = current_hash
          current_hash = calcBlockHash(current_hash + sibling_nodes[i][1].data) #Hashed
          
          print("RIGHT");
          print("node range ",sibling_nodes[i][1].range)
          if(sibling_nodes[i][1].parent is not None): 
            print("parent hash ",sibling_nodes[i][1].parent.data, current_hash)
            print("left hash ",sibling_nodes[i][1].parent.left.data, temp)
            print("right hash ",sibling_nodes[i][1].parent.right.data, sibling_nodes[i][1].data)
          
        else:
          current_hash = calcBlockHash(current_hash + sibling_nodes[i][1]) #Hashed
    return current_hash # will return the updated hash 


In [ ]:
import string
import random
 
N = BLOCK_SIZE
blocks_ = []

for i in range(10):
  res = ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))
  blocks_.append(res)


print(len(blocks_))

10


Old Version


In [ ]:
# Encryption Test
enc_blocks = []
for i in range(len(blocks_)):
  tmp = encrypt_block(blocks_[i])
  enc_blocks.append(tmp)


# Decryption Test

dec_blocks = []
for i in range(len(enc_blocks)):
  tmp = decrypt_block(enc_blocks[i])
  dec_blocks.append(tmp)


print(type(blocks_[0]), blocks_)
print(type(enc_blocks[0]), enc_blocks)
print(type(dec_blocks[0]), dec_blocks)


block_hashes = calcFileBlockHashes(enc_blocks)
print(type(block_hashes[0]), block_hashes)


# res_dec = decrypt_block(t1)
# print(type(res_dec), res_deca)

# fh = calcBlockHash(t1)

# print(type(fh),fh)

# fbh = calcFileBlockHashes([t1, t2]);
# print(type(fbh[0]))
# print(fbh)

<class 'str'> ['...']
<class 'str'> ['...']
<class 'str'> ['...']
<class 'bytes'> [b'\xe8\xfe{T|\xa9`,*\xd3\x11\xf8\x83\x02\xc6\xc7EvA\x92\xd8\x13\xfd\x17w\xb5sn\xed\xb5\x92\x08', b'\x17\xe9\x8f\x1f\x08W@b\xb5\x86\xfc\xf0;A\x1d\xadz\xfa\x8c\xaeh\x93\x1b\xf1\x01\x0f\xc3W\x01\xdai\xa0', b'\xc6?\xc7\x0eY\x9f\xb3\x88\xd9\x1b\xaf\xbbX\xeb\xbc\xd3\x0bx\x98\x16\x02\xde?#\x1c\x86g`5\xf9\x92\x8a', b"\x0c\x03\xa1o\xaf\x8dy\x92\x18@\x9cy'\xde\x1d&=\xd6\xf74\xf0L\xf1\xc0\x93\\\xea\x7f\x9fL\x94W", b'\xf1\x97"\x98\x9a\x1c=\xd5-i\x80\xc7\x17#\xe9(9\x80d\x9f\xb0fgyZ\xbd\xcf]}\xb2\xddE', b'$nY\xb3\xf6S\xc0\x17\x9f\xfe\xb1ID\x90*Z\xab\xb5E}_zO1\x99\x9f\xd2\x0c\xfa\xde\xbb\xe0', b'\xe4A\xfd\x89\xe6\xbb\xea\xaf\x01a\x82\xc5\xe4\xb9\xccEQ\x8b\xe0\xf8G\x90\x9c\xc6X\xa0k\xfe"\xebn#', b'\xb4\xfc\xdc\xb7\x91\x07\x11\xbf\x91\xa6t1\xc5\xadAw\xb6\xb7\x06\xe1\xbd\xff\xda\xbd\xc7~H\x08\xee\x1c\x0c\xe0', b'\xbb\x8a\x9a\xd1\xd1e\xfb2\xdeuKdk\x0b\x9a\x17\xeaBp\x82\xb8STH#\x84&\xd5\x18z\xe5E', b'\x88\xd8\xb6\xdf\xf1\xa

In [ ]:
# def calcBlockHash_(block):
#   hasher = hashlib.sha256()
#   hasher.update(str(block).encode())
#   return hasher.hexdigest()

In [ ]:
# def calcFileBlockHashes_(encrypted_file_blocks):
#   hashed_blocks = []
#   for i in encrypted_file_blocks:
#     hashed_blocks.append(calcBlockHash(i))
#   return hashed_blocks

In [ ]:
# BLOCK_LEN = 16000 #16 kb

In [ ]:
# !pip3 install cryptography

In [ ]:
# from cryptography.fernet import Fernet


# key = Fernet.generate_key()

# # Instance the Fernet class with the key
# fernet = Fernet(key)

# def encrypt_block_(text):
#   encMessage = fernet.encrypt(text.encode())
#   return encMessage.decode("utf-8")
  

# def decrypt_block_(text):
#   decMessage = fernet.decrypt(text).decode()
#   return decMessage



In [ ]:
# from cryptography.fernet import Fernet
 
# # we will be encrypting the below string.
# message = "hello geeks"
 
# # generate a key for encryption and decryption
# # You can use fernet to generate
# # the key or use random key generator
# # here I'm using fernet to generate key
 
# key = Fernet.generate_key()
 
# # Instance the Fernet class with the key
 
# fernet = Fernet(key)
 
# # then use the Fernet class instance
# # to encrypt the string string must
# # be encoded to byte string before encryption
# encMessage = fernet.encrypt(res.encode())
 
# print("original string: ", res)
# print("encrypted string: ", type(encMessage.decode("utf-8")), encMessage)
 
# # decrypt the encrypted string with the
# # Fernet instance of the key,
# # that was used for encrypting the string
# # encoded byte string is returned by decrypt method,
# # so decode it to string with decode methods
# decMessage = fernet.decrypt(encMessage).decode()
 
# print("decrypted string: ", decMessage)

In [ ]:

# def calculateBlockHash_(block):
#   hasher = hashlib.sha256()
#   hasher.update(str(block).encode())
#   return hasher.hexdigest()

In [ ]:
import string
import random
 
N = BLOCK_SIZE

res = ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))
res2 = ''.join(random.choices(string.ascii_uppercase + string.digits, k=N))

print(len(res),len(res2))

In [ ]:
print(res)
a = encrypt_block(res)
print(type(a),a)
b = decrypt_block(a)
print(type(b), b)


k = calcBlockHash(a)
print(type(k), k)

In [ ]:
# f1 = calculateBlockHash_(t1)
# print(type(f1))